# COGS 108 - Data Checkpoint

# Names

- Thuy Nguyen
- Tony Hu
- Raul Vargas
- Kristen Corpuz
- Megan Li

<a id='research_question'></a>
# Research Question

How did the pandemic affect multiplayer game (games with social interaction) games available on steam:
- How did the pandemic in 2020 affect player count of the most popular multiplayer games on steam versus the most popular single single player games? 
- Potential: How did the pandemic affect the types of games that were released before the pandemic versus after the pandemic?

# Dataset(s)

**Dataset 1 Name: Application Information (csv file)**

- Link to the dataset: https://data.mendeley.com/datasets/ycy3sy3vj2/1
- Number of observations: 2000
- Note that there are 1963 games in the dataset out of the 2000 applications before data cleaning

This dataset has 2000 rows and 5 columns, containing the game ID in the first column and application type, name, release date, and free to play values in the rest of the four columns.

**Dataset 2 Name: Application Genres (csv file)**
- Link to the dataset: https://data.mendeley.com/datasets/ycy3sy3vj2/1
- Number of observations: 2000

This dataset has 2000 rows and 2 columns, containing the game ID in the first column and a list of genres corresponding to each game in the second column.

**Dataset 3 Name: Application Tags (csv file)**
- Link to the dataset: https://data.mendeley.com/datasets/ycy3sy3vj2/1
- Number of observations: 2000

This dataset has 2000 rows and 2 columns, containing the game ID in the first column and a list of game tags corresponding to each game in the second column.

**Dataset 4 Name: Player Count History Part 1 and Player Count History Part 2 (zip files)**
- Link to the dataset: https://data.mendeley.com/datasets/ycy3sy3vj2/1
- Number of observations: 2000 individual csv files corresponding to each game ID

Each Player Count History zip file contains 2000 csv files for each game. Each game’s file has 280,225 rows and 2 columns, containing the date and time in the first column in 5-minute intervals starting from 12/14/2017 12:00 AM to 8/12/2020 11:55 PM and player count for the corresponding date and time in the second column.

**Merging the data**

We plan to merge datasets 1, 2, and 3 on the game ID using an inner merge and only keeping the rows that exist in all 3 datasets after individually pruning rows in each dataset. After cleaning the top 5 games for selected genres’ (such as single player and multiplayer) player count history files to store monthly averaged player counts starting from January 2019 to August 2020, we may merge this information with our previously combined dataset using the game ID.

# Setup

In [1]:
# importing libraries we might need later
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import seaborn as sns

In [2]:

# Read applicationGenres.csv into dataframe storing only IDs with genres
with open('./datasets/applicationGenres.csv') as f:
    genresDict = {} # Store distinct genres
    IDs = []
    genres = []
    for line in f:
        if ',' in line:
            cleanLine = line.strip().split(',')
            IDs.append(cleanLine[0])
            genres.append(cleanLine[1:]) #genres.append(','.join(cleanLine[1:])) converts list to string
            for genre in cleanLine[1:]: genresDict.setdefault(genre)
df_genre = pd.DataFrame({'ID':IDs, 'Genres':genres},)
df_genre["ID"] = pd.to_numeric(df_genre["ID"])
df_genre

,ID,Genres
0,578080,"[Action, Adventure, Massively Multiplayer]"
1,570,"[Action, Free to Play, Strategy]"
2,730,"[Action, Free to Play]"
3,359550,[Action]
4,271590,"[Action, Adventure]"
...,...,...
1827,64000,[Strategy]
1828,227860,"[Indie, Strategy]"
1829,429050,"[Action, Indie, Simulation, Early Access]"
1830,209650,[Action]


Using one-hot encoding to separate genres into their own columns instead of having them listed as list of genres for every observation like the table above. Every genre column will be marked with 1 if it is present in the original list of genres for each observation.

In [3]:
df_genre = df_genre.drop('Genres', 1).join(df_genre.Genres.str.join('|').str.get_dummies())

In [4]:
print(df_genre.shape)
df_genre

(1832, 27)


,ID,60,Action,Adventure,Animation & Modeling,Audio Production,Casual,Design & Illustration,Early Access,Education,...,Racing,Sexual Content,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Violent,Web Publishing
0,578080,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,570,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,730,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,359550,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,271590,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1827,64000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1828,227860,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1829,429050,0,1,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
1830,209650,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Next, we will look at all columns that has at least one observation with a value 1 for df_genres.

In [5]:
(df_genre.loc[:, df_genre.columns != 'ID'] > 0).any()

60                       True
Action                   True
Adventure                True
Animation & Modeling     True
Audio Production         True
Casual                   True
Design & Illustration    True
Early Access             True
Education                True
Free to Play             True
Gore                     True
Indie                    True
Massively Multiplayer    True
Nudity                   True
Photo Editing            True
RPG                      True
Racing                   True
Sexual Content           True
Simulation               True
Software Training        True
Sports                   True
Strategy                 True
Utilities                True
Video Production         True
Violent                  True
Web Publishing           True
dtype: bool

It seems like all columns of different genres have at least one observation that has a value of 1. However, we will be dropping the column labeled '60' due its ambiguity and legen-related explanation from the source.

In [6]:
df_genre.drop('60', inplace=True, axis=1)
print(list(df_genre.columns))

['ID', 'Action', 'Adventure', 'Animation & Modeling', 'Audio Production', 'Casual', 'Design & Illustration', 'Early Access', 'Education', 'Free to Play', 'Gore', 'Indie', 'Massively Multiplayer', 'Nudity', 'Photo Editing', 'RPG', 'Racing', 'Sexual Content', 'Simulation', 'Software Training', 'Sports', 'Strategy', 'Utilities', 'Video Production', 'Violent', 'Web Publishing']


In [7]:
df_genre.shape

(1832, 26)

Next we will be importing data from the file "applicationInformation.csv".

In [8]:
# Get the applicationInformation.csv and remove rows that are not games and empty
df_info = pd.read_csv("./datasets/applicationInformation.csv", engine="python")
# renaming the appid column to ID, matching the ID column for df_genre
df_info = df_info.rename(columns={'appid': 'ID'})
print(df_info.shape)
df_info

(2000, 5)


,ID,type,name,releasedate,freetoplay
0,578080,game,PLAYERUNKNOWN'S BATTLEGROUNDS,21-Dec-17,0.0
1,570,game,Dota 2,9-Jul-13,1.0
2,730,game,Counter-Strike: Global Offensive,21-Aug-12,1.0
3,622590,NaN,PLAYERUNKNOWN'S BATTLEGROUNDS (Test Server),NaN,NaN
4,359550,game,Tom Clancy's Rainbow Six Siege,1-Dec-15,0.0
...,...,...,...,...,...
1995,34000,advertising,Football Manager 2010,29-Oct-09,1.0
1996,202480,NaN,Skyrim Creation Kit,NaN,NaN
1997,429050,game,Feed and Grow: Fish,8-Jan-16,0.0
1998,209650,game,Call of Duty: Advanced Warfare,3-Nov-14,0.0


Since we will only be exploring data about video games, we will be dropping any observations with a non-game for the "type" column

In [9]:
df_info = df_info[df_info.type == 'game']
print(df_info.shape)
df_info

(1851, 5)


,ID,type,name,releasedate,freetoplay
0,578080,game,PLAYERUNKNOWN'S BATTLEGROUNDS,21-Dec-17,0.0
1,570,game,Dota 2,9-Jul-13,1.0
2,730,game,Counter-Strike: Global Offensive,21-Aug-12,1.0
4,359550,game,Tom Clancy's Rainbow Six Siege,1-Dec-15,0.0
5,271590,game,Grand Theft Auto V,13-Apr-15,0.0
...,...,...,...,...,...
1993,64000,game,Men of War: Assault Squad,24-Feb-11,0.0
1994,227860,game,Castle Story,17-Aug-17,0.0
1997,429050,game,Feed and Grow: Fish,8-Jan-16,0.0
1998,209650,game,Call of Duty: Advanced Warfare,3-Nov-14,0.0


Next we will be importing data from the file "applicationTags.csv".

In [10]:
with open('./datasets/applicationTags.csv') as f:
    tagDict = {} # Store distinct genres
    IDs = []
    tags = []
    for line in f:
        if ',' in line:
            cleanLine = line.strip().split(',')
            IDs.append(cleanLine[0])
            tags.append(cleanLine[1:]) 
            for tag in cleanLine[1:]: tagDict.setdefault(tag)
df_tags = pd.DataFrame({'ID':IDs, 'Tags':tags},)
df_tags["ID"] = pd.to_numeric(df_tags["ID"])

Again, using one-hot encoding to separate tags into their own columns.

In [11]:
df_tags = df_tags.drop('Tags', 1).join(df_tags.Tags.str.join('|').str.get_dummies())
print(df_tags.shape)
df_tags

(1945, 340)


,ID,1980s,1990's,2.5D,2D,2D Fighter,3D,3D Platformer,3D Vision,4 Player Local,...,Warhammer 40K,Web Publishing,Werewolves,Western,Word Game,World War I,World War II,Wrestling,Zombies,e-sports
0,578080,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,570,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,730,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,359550,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,271590,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1940,64000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1941,227860,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1942,429050,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1943,209650,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Since the ID column is the consistent way of identifying the applications listed, we will merge all 3 datasets into 1 joint dataset to provide more accesibility for data analysis and exploration. "df_genre" has the lowest amount of observations, therefore, we will use its amount of observations as the lower bound for merging the datasets.

In [12]:
df_genre_tags = pd.merge(df_genre, df_tags,on='ID') # shape = (1825, 365) after merging
combined_df = pd.merge(df_info, df_genre_tags, on='ID') # shape = (1803, 369) after merging
combined_df.shape

(1803, 369)

# Data Cleaning

There are a lot of columns on our final dataset merge. Let's take a look at what those columns are and decide which are most valuable to our research question.

In [13]:
print(list(combined_df.columns))

['ID', 'type', 'name', 'releasedate', 'freetoplay', 'Action_x', 'Adventure_x', 'Animation & Modeling_x', 'Audio Production_x', 'Casual_x', 'Design & Illustration_x', 'Early Access_x', 'Education_x', 'Free to Play_x', 'Gore_x', 'Indie_x', 'Massively Multiplayer_x', 'Nudity_x', 'Photo Editing_x', 'RPG_x', 'Racing_x', 'Sexual Content_x', 'Simulation_x', 'Software Training_x', 'Sports_x', 'Strategy_x', 'Utilities_x', 'Video Production_x', 'Violent_x', 'Web Publishing_x', '1980s', "1990's", '2.5D', '2D', '2D Fighter', '3D', '3D Platformer', '3D Vision', '4 Player Local', '4X', '6DOF', 'Abstract', 'Action_y', 'Action RPG', 'Action-Adventure', 'Addictive', 'Adventure_y', 'Agriculture', 'Aliens', 'Alternate History', 'America', 'Animation & Modeling_y', 'Anime', 'Arcade', 'Arena Shooter', 'Artificial Intelligence', 'Assassin', 'Asynchronous Multiplayer', 'Atmospheric', 'Audio Production_y', 'Base-Building', 'Based On A Novel', 'Basketball', 'Batman', 'Battle Royale', "Beat 'em up", 'Beautiful'

Looking at all the genres and tags we got from combining the 3 data sets, we decided to stick to only ['ID', 'type', 'name', 'releasedate', 'freetoplay', 'Free to Play_x', 'Massively Multiplayer_x', 'Asynchronous Multiplayer', 'Casual_y', 'Co-op', 'Co-op Campaign', 'Competitive', 'Free to Play_y', 'Local Co-Op', 'Local Multiplayer', 'Massively Multiplayer_y', 'Multiplayer', 'Online Co-Op', 'Party-Based RPG', 'PvE', 'PvP', 'RPG_x', 'RPG_y', 'Replay Value', 'Singleplayer']. The reason we chose to do so is because we believe these features may be directly related to our research questions or may be confounders to the variables we may be investigating for our research questions.

In [14]:
combined_reduced = combined_df[['ID', 'type', 'name', 'releasedate', 'freetoplay', 'Free to Play_x', 'Massively Multiplayer_x', 'Asynchronous Multiplayer', 'Casual_y', 'Co-op', 'Co-op Campaign', 'Competitive', 'Free to Play_y', 'Local Co-Op', 'Local Multiplayer', 'Massively Multiplayer_y', 'Multiplayer', 'Online Co-Op', 'Party-Based RPG', 'PvE', 'PvP', 'RPG_x', 'RPG_y', 'Replay Value', 'Singleplayer']]
#print(combined_reduced['Free to Play_y'])
combined_reduced

,ID,type,name,releasedate,freetoplay,Free to Play_x,Massively Multiplayer_x,Asynchronous Multiplayer,Casual_y,Co-op,...,Massively Multiplayer_y,Multiplayer,Online Co-Op,Party-Based RPG,PvE,PvP,RPG_x,RPG_y,Replay Value,Singleplayer
0,578080,game,PLAYERUNKNOWN'S BATTLEGROUNDS,21-Dec-17,0.0,0,1,0,0,1,...,0,1,1,0,0,1,0,0,0,0
1,570,game,Dota 2,9-Jul-13,1.0,1,0,0,0,1,...,0,1,1,0,0,1,0,1,1,0
2,730,game,Counter-Strike: Global Offensive,21-Aug-12,1.0,1,0,0,0,1,...,0,1,1,0,0,1,0,0,0,0
3,359550,game,Tom Clancy's Rainbow Six Siege,1-Dec-15,0.0,0,0,0,1,1,...,1,1,1,0,0,0,0,0,0,1
4,271590,game,Grand Theft Auto V,13-Apr-15,0.0,0,0,0,0,1,...,0,1,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1798,64000,game,Men of War: Assault Squad,24-Feb-11,0.0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1
1799,227860,game,Castle Story,17-Aug-17,0.0,0,0,0,1,1,...,0,1,0,0,0,0,0,0,0,1
1800,429050,game,Feed and Grow: Fish,8-Jan-16,0.0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1
1801,209650,game,Call of Duty: Advanced Warfare,3-Nov-14,0.0,0,0,0,0,1,...,0,1,1,0,0,1,0,0,0,1


Looking at the remaining columns, we decide to drop the "type" column due to it having only one type which is "game." Duplicated columns resulted from the merge will also be joined.

In [15]:
combined_reduced.drop('type', inplace=True, axis=1) # dropping type column

/Users/thuytee/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [16]:
var_missing = combined_reduced.isna().any().sum()
combined_reduced[combined_reduced['releasedate'].isna()]

,ID,name,releasedate,freetoplay,Free to Play_x,Massively Multiplayer_x,Asynchronous Multiplayer,Casual_y,Co-op,Co-op Campaign,...,Massively Multiplayer_y,Multiplayer,Online Co-Op,Party-Based RPG,PvE,PvP,RPG_x,RPG_y,Replay Value,Singleplayer
126,219740,Don't Starve,NaN,0.0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,1
823,272230,Sub Rosa,NaN,0.0,0,0,0,1,1,0,...,1,1,1,0,0,0,0,0,0,0
829,224540,Ace of Spades,NaN,1.0,0,0,0,1,1,0,...,1,1,0,0,0,0,0,0,1,0
938,223850,3DMark,NaN,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
943,212480,Sonic & All-Stars Racing Transformed,NaN,0.0,0,0,0,0,1,0,...,0,1,1,0,0,0,0,0,1,1
978,237110,Mortal Kombat Komplete Edition,NaN,0.0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,1
1234,564310,Serious Sam Fusion 2017 (beta),NaN,0.0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,1
1268,214560,Mark of the Ninja,NaN,1.0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,1
1271,202170,Sleeping Dogs�,NaN,1.0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,1
1570,39550,SpellForce 2 - Anniversary Edition,NaN,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,1


In [17]:
combined_reduced['freetoplay'] = combined_reduced['freetoplay'].astype('bool')
combined_reduced['Free to Play_x'] = combined_reduced['Free to Play_x'].astype('bool')
combined_reduced['Free to Play_y'] = combined_reduced['Free to Play_y'].astype('bool')
combined_reduced.assign(free_to_play = (combined_reduced['freetoplay'] | combined_reduced['Free to Play_x'] | combined_reduced['Free to Play_y']))
#combined_reduced.loc['free_to_play']

<ipython-input-17-ae45e1d7a509>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_reduced['freetoplay'] = combined_reduced['freetoplay'].astype('bool')
<ipython-input-17-ae45e1d7a509>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_reduced['Free to Play_x'] = combined_reduced['Free to Play_x'].astype('bool')
<ipython-input-17-ae45e1d7a509>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



,ID,name,releasedate,freetoplay,Free to Play_x,Massively Multiplayer_x,Asynchronous Multiplayer,Casual_y,Co-op,Co-op Campaign,...,Multiplayer,Online Co-Op,Party-Based RPG,PvE,PvP,RPG_x,RPG_y,Replay Value,Singleplayer,free_to_play
0,578080,PLAYERUNKNOWN'S BATTLEGROUNDS,21-Dec-17,False,False,1,0,0,1,0,...,1,1,0,0,1,0,0,0,0,False
1,570,Dota 2,9-Jul-13,True,True,0,0,0,1,0,...,1,1,0,0,1,0,1,1,0,True
2,730,Counter-Strike: Global Offensive,21-Aug-12,True,True,0,0,0,1,0,...,1,1,0,0,1,0,0,0,0,True
3,359550,Tom Clancy's Rainbow Six Siege,1-Dec-15,False,False,0,0,1,1,0,...,1,1,0,0,0,0,0,0,1,False
4,271590,Grand Theft Auto V,13-Apr-15,False,False,0,0,0,1,0,...,1,0,0,0,0,0,1,0,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1798,64000,Men of War: Assault Squad,24-Feb-11,False,False,0,0,0,1,0,...,1,0,0,0,0,0,0,0,1,False
1799,227860,Castle Story,17-Aug-17,False,False,0,0,1,1,0,...,1,0,0,0,0,0,0,0,1,False
1800,429050,Feed and Grow: Fish,8-Jan-16,False,False,0,0,0,1,0,...,1,0,0,0,0,0,0,0,1,True
1801,209650,Call of Duty: Advanced Warfare,3-Nov-14,False,False,0,0,0,1,0,...,1,1,0,0,1,0,0,0,1,False


In [20]:
#combined_reduced['free_to_play'].replace({'True': 1}, {'False': 0}, inplace=True)
#combined_reduced.drop(['freetoplay', 'Free to Play_x', 'Free to Play_y'], inplace=True, axis=1)
#combined_reduced.drop('Free to Play_x', inplace=True, axis=1)
#combined_reduced.drop('Free to Play_y', inplace=True, axis=1)

KeyError: 'free_to_play'